# Face Mask Detection | YOLOv5


The purpose of this notebook is to create a Deep Learning model that identifies whether or not someone is wearing a mask. My model is based on YOLO's object detection algorithm, and I'm using the dataset from kaggle's website, which is linked below. A few changes have been made to the dataset's structure. Below is the structure of the dataset that contains 6900 image data and their labels in YOLOv5 format.
```
.
└── raw_data
    ├── classes.txt
    ├── images
    └── labels
```

YOLOv5 : https://github.com/ultralytics/yolov5

Dataset : https://www.kaggle.com/datasets/parot99/face-mask-detection-yolo-darknet-format

**The dataset consists of two classes:**
- with_mask
- without_mask


**The dataset is split into 3 sets:**
- Train : 5800 images
- Validation : 1000 images
- Test : 100 images


In [1]:
import warnings
warnings.filterwarnings('ignore')

import torch 
import os
import shutil 
import random
from PIL import Image

### Data preparation

In [2]:
cwd = os.getcwd()
print(f'Current Working Directory : {cwd}')

Current Working Directory : /Users/parisa/Face-Mask-Detection-YOLOv5


In [3]:
# Verify that each image has a label

raw_images_dir = os.path.join(cwd, 'raw_data/images')
raw_labels_dir = os.path.join(cwd, 'raw_data/labels')

for file in os.listdir(raw_images_dir):
    file_name, file_extension = os.path.splitext(file)
    label = file_name + '.txt'
    if label not in os.listdir(raw_labels_dir):
        print(f'Label For Image "{file_name}" Is Not Found')

To split image data (raw_data) into train, test, and validation sets, I created new directories using the "create_dir" function, then moved each image and label into its final location. Below is the structure of the dataset directory we are going to use for training our model. 
```
.
└── dataset
    ├── images
    │   ├── train
    │   ├── test
    │   └── val
    └── labels
        ├── train
        ├── test
        └── val
```

In [4]:
def create_dir(root_dir, sub_dir):
    try:
        path = os.path.join(root_dir, sub_dir)
        os.makedirs(path)
        print(f'Directory "{path}" Created')
    except FileExistsError:
        print('Directory Already Exists') 

In [5]:
sub_dirs = [
    'dataset/images/train', 
    'dataset/images/val', 
    'dataset/images/test',
    'dataset/labels/train', 
    'dataset/labels/val',
    'dataset/labels/test'
]

for sub_dir in sub_dirs:
    create_dir(cwd, sub_dir)

Directory "/Users/parisa/Face-Mask-Detection-YOLOv5/dataset/images/train" Created
Directory "/Users/parisa/Face-Mask-Detection-YOLOv5/dataset/images/val" Created
Directory "/Users/parisa/Face-Mask-Detection-YOLOv5/dataset/images/test" Created
Directory "/Users/parisa/Face-Mask-Detection-YOLOv5/dataset/labels/train" Created
Directory "/Users/parisa/Face-Mask-Detection-YOLOv5/dataset/labels/val" Created
Directory "/Users/parisa/Face-Mask-Detection-YOLOv5/dataset/labels/test" Created


In [6]:
train_size = 5800
val_size = 1000
test_size = 100

train_samples = random.sample(os.listdir(raw_images_dir), train_size)
for sample in train_samples:
    os.listdir(raw_images_dir).remove(sample)
    
val_samples = random.sample(os.listdir(raw_images_dir), val_size)
for sample in val_samples:
    os.listdir(raw_images_dir).remove(sample)
    
test_samples = random.sample(os.listdir(raw_images_dir), test_size)

In [7]:
# Move images and labels of training set 

train_image_path = os.path.join(cwd, 'dataset/images/train/')
train_labels_path = os.path.join(cwd, 'dataset/labels/train/')

for sample in train_samples:
    # move image
    image = Image.open(os.path.join(raw_images_dir, sample))
    image = image.convert('RGB')
    image.save(train_image_path + sample, format="jpeg") 
    os.listdir(raw_images_dir).remove(sample)
    # move label
    sample_name, sample_extension = os.path.splitext(sample)
    label = os.path.join(raw_labels_dir, sample_name + '.txt')
    shutil.copy(label, os.path.join(train_labels_path, sample_name + '.txt'))

In [8]:
# Move images and labels of validation set

val_image_path = os.path.join(cwd, 'dataset/images/val/')
val_labels_path = os.path.join(cwd, 'dataset/labels/val/')

for sample in val_samples:
    # move image
    image = Image.open(os.path.join(raw_images_dir, sample))
    image = image.convert('RGB')
    image.save(val_image_path + sample, format="jpeg") 
    os.listdir(raw_images_dir).remove(sample)
    # move label
    sample_name, sample_extension = os.path.splitext(sample)
    label = os.path.join(raw_labels_dir, sample_name + '.txt')
    shutil.copy(label, os.path.join(val_labels_path, sample_name + '.txt'))

In [9]:
# Move images and labels of test set

test_image_path = os.path.join(cwd, 'dataset/images/test/')
test_labels_path = os.path.join(cwd, 'dataset/labels/test/')

for sample in test_samples:
    # move image
    image = Image.open(os.path.join(raw_images_dir, sample))
    image = image.convert('RGB')
    image.save(test_image_path + sample, format="jpeg") 
    os.listdir(raw_images_dir).remove(sample)
    # move label
    sample_name, sample_extension = os.path.splitext(sample)
    label = os.path.join(raw_labels_dir, sample_name + '.txt')
    shutil.copy(label, os.path.join(test_labels_path, sample_name + '.txt'))

### Model training